# BingSearch using Kernel 

This notebook explains how to integrate Bing Search with the Semantic Kernel  to get the latest information from the internet.

To use Bing Search you simply need a Bing Search API key. You can get the API key by creating a [Bing Search resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) in Azure. 

To learn more read the following [documentation](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/overview).


Prepare a Semantic Kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-rc3"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.0-rc3"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.TemplateEngine;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, "model-id", azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Add the following namespaces to use the Bing Search connector.

In [ ]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

Enter your Bing Search Key in BING_KEY using `InteractiveKernel` method as introduced in [`.NET Interactive`](https://github.com/dotnet/interactive/blob/main/docs/kernels-overview.md)

In [ ]:
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

string BING_KEY = (await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search Key")).GetClearTextPassword();

Below are some examples on how [`WebSearchEnginePlugin`](../src/Plugins/Plugins.Web/WebSearchEnginePlugin.cs) can be used in SK. 

In [ ]:
private static async Task Example1Async(Microsoft.SemanticKernel.Kernel kernel)
{
        Console.WriteLine("Example 1");

        // Run 
        var question = "What is quantum tunnelling";
        var function = kernel.Plugins["bing"]["search"];
        var bingResult = await kernel.InvokeAsync(function, new KernelArguments(question));

        Console.WriteLine(question);
        Console.WriteLine("----");
        Console.WriteLine(bingResult);
        Console.WriteLine();

        /* OUTPUT:

            What is quantum tunnelling
            ----
            In physics, quantum tunnelling, barrier penetration, or simply tunnelling is a quantum mechanical
            phenomenon in which an object such as an electron or atom passes through a potential energy 
            barrier that, according to classical mechanics, the object does not have sufficient energy to
            enter or surmount.
            
       */
}

In [ ]:
private static async Task Example2Async(Microsoft.SemanticKernel.Kernel kernel)
{
    Console.WriteLine("Example 2");

    //The following function only works in interactive notebooks
    string question = await InteractiveKernel.GetInputAsync("Please ask your question"); 

    var function = kernel.Plugins["bing"]["search"];
    var bingResult = await kernel.InvokeAsync(function, new KernelArguments(question));

    Console.WriteLine(bingResult);
}

Just uncomment the examples to run them in the following code:

In [ ]:
#pragma warning disable SKEXP0054

// Load Bing plugin
var bingConnector = new BingConnector(BING_KEY);

kernel.ImportPluginFromObject(new WebSearchEnginePlugin(bingConnector), "bing");

await Example1Async(kernel);
await Example2Async(kernel);